### 경기도 공원 분석

In [5]:
import json, folium, warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

- 경기도 공원 데이터

In [12]:
park = pd.read_csv('data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
park.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17525 entries, 0 to 17524
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   관리번호          17525 non-null  object 
 1   공원명           17525 non-null  object 
 2   공원구분          17525 non-null  object 
 3   소재지도로명주소      3692 non-null   object 
 4   소재지지번주소       17335 non-null  object 
 5   위도            17513 non-null  float64
 6   경도            17514 non-null  float64
 7   공원면적          17525 non-null  float64
 8   공원보유시설(운동시설)  4821 non-null   object 
 9   공원보유시설(유희시설)  6200 non-null   object 
 10  공원보유시설(편익시설)  4842 non-null   object 
 11  공원보유시설(교양시설)  805 non-null    object 
 12  공원보유시설(기타시설)  2577 non-null   object 
 13  지정고시일         14451 non-null  object 
 14  관리기관명         16033 non-null  object 
 15  전화번호          16343 non-null  object 
 16  데이터기준일자       17525 non-null  object 
 17  제공기관코드        17525 non-null  object 
 18  제공기관명         17525 non-nu

In [13]:
# 필요한 컬럼만 가져오기
park = park[['공원명','소재지지번주소','공원면적']]
park.dropna(how='any', inplace=True)
park.head()

,공원명,소재지지번주소,공원면적
0,녹색소공원,서울특별시 용산구 효창동 287-6,1569.9
23,강변공원,서울특별시 용산구 이촌동 408,1766.0
24,솔밭어린이공원,서울특별시 용산구 한강로1가 30-3,2090.0
25,신창동어린이공원,서울특별시 용산구 신창동 100,257.0
26,서빙고공원,서울특별시 용산구 용산동5가 24-2,11879.0


In [14]:
# 경기도 데이터만 추출하기
park = park[park.소재지지번주소.str.contains('경기도')]
park.head()

,공원명,소재지지번주소,공원면적
212,꿈나무공원,경기도 이천시 중리동 15,2526.0
213,남천공원,경기도 이천시 중리동 210,3830.0
214,향교공원,경기도 이천시 창전동 408,1757.0
215,개나리공원,경기도 이천시 창전동 414-13,1650.0
216,달맞이공원,경기도 이천시 창전동 423,1650.0


- 경기도 인구 데이터

In [15]:
pop = pd.read_csv('../02.Pandas/data/경기도_주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수']]
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]

In [16]:
# 수원시, 용인시 등 구가 있는 시는 제거
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


- 데이터 프레임 합성 방향
    - 인구데이터 + 공원데이터 합성
    - 합성된 데이터와 지도데이터 조인

In [17]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ' '.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시 덕양구
31,구,경기도 고양시 일산동구,296590,고양시 일산동구
44,구,경기도 고양시 일산서구,290738,고양시 일산서구
56,시군,경기도 과천시,77775,과천시


In [18]:
park['시군구'] = ['-'] * len(park)
park.head()

,공원명,소재지지번주소,공원면적,시군구
212,꿈나무공원,경기도 이천시 중리동 15,2526.0,-
213,남천공원,경기도 이천시 중리동 210,3830.0,-
214,향교공원,경기도 이천시 창전동 408,1757.0,-
215,개나리공원,경기도 이천시 창전동 414-13,1650.0,-
216,달맞이공원,경기도 이천시 창전동 423,1650.0,-


In [19]:
# 방법 1. 소재지지번주소 열에서 시군구만 추출하기
for i in park.index:
    for sigungu in pop.시군구:
        if sigungu in park.소재지지번주소[i]:
            park.시군구[i] = sigungu
            break

In [20]:
# 방법 2
park['시군구2'] = park.소재지지번주소.apply(lambda x: 
                    x.split()[1] if x.split()[2][-1] != '구' else ' '.join(x.split()[1:3]))
park.tail()

,공원명,소재지지번주소,공원면적,시군구,시군구2
17025,제90호 어린이공원(풀향기),경기도 용인시 기흥구 중동 1083,1500.0,용인시 기흥구,용인시 기흥구
17258,청석공원,경기도 광주시 경안동 242,193530.0,광주시,광주시
17259,경안근린공원,경기도 광주시 경안동 168-1,83095.0,광주시,광주시
17260,삼리어린이공원,경기도 광주시 곤지암읍 삼리 131-1,1493.0,광주시,광주시
17261,중대물빛공원,경기도 광주시 중대동 246-2,230932.0,광주시,광주시


In [21]:
print(park.시군구.isna().sum())
park.tail()

0


,공원명,소재지지번주소,공원면적,시군구,시군구2
17025,제90호 어린이공원(풀향기),경기도 용인시 기흥구 중동 1083,1500.0,용인시 기흥구,용인시 기흥구
17258,청석공원,경기도 광주시 경안동 242,193530.0,광주시,광주시
17259,경안근린공원,경기도 광주시 경안동 168-1,83095.0,광주시,광주시
17260,삼리어린이공원,경기도 광주시 곤지암읍 삼리 131-1,1493.0,광주시,광주시
17261,중대물빛공원,경기도 광주시 중대동 246-2,230932.0,광주시,광주시


In [12]:
df = park.pivot_table('공원면적', '시군구', aggfunc='sum')
df.head()

,공원면적
시군구,
-,2034792.0
가평군,511674.5
고양시 덕양구,3287291.4
고양시 일산동구,2722980.0
고양시 일산서구,922486.9


In [13]:
# df는 index, pop은 '시군구' 컬럼으로 조인
df = df.merge(pop, left_index=True, right_on='시군구')
df['인당 공원면적'] = (df.공원면적 / df['총 인구수']).round(1)
df.head()

,공원면적,행정구역구분명,행정구역명,총 인구수,시군구,인당 공원면적
1,511674.5,시군,경기도 가평군,62197,가평군,8.2
9,3287291.4,구,경기도 고양시 덕양구,487874,고양시 덕양구,6.7
31,2722980.0,구,경기도 고양시 일산동구,296590,고양시 일산동구,9.2
44,922486.9,구,경기도 고양시 일산서구,290738,고양시 일산서구,3.2
56,315666.0,시군,경기도 과천시,77775,과천시,4.1


In [14]:
# 지도 데이터와 조인하기 위해서 시군구 열에서 공백을 제거하고 인덱스로 만들어 줌
df.시군구 = df.시군구.str.replace(' ','')
df.set_index('시군구', inplace=True)
df.head()

,공원면적,행정구역구분명,행정구역명,총 인구수,인당 공원면적
시군구,,,,,
가평군,511674.5,시군,경기도 가평군,62197,8.2
고양시덕양구,3287291.4,구,경기도 고양시 덕양구,487874,6.7
고양시일산동구,2722980.0,구,경기도 고양시 일산동구,296590,9.2
고양시일산서구,922486.9,구,경기도 고양시 일산서구,290738,3.2
과천시,315666.0,시군,경기도 과천시,77775,4.1


In [15]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

In [16]:
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [17]:
sigungu_dict = get_text_location(geo_data)

map = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.공원면적,      # 단계구분도로 보여줄 데이터
    columns=[df.index, df.공원면적],    # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',        # Colormap
    key_on='properties.name'    # 지도에서 조인할 항목
).add_to(map)
for sigungu_name in df.index:
    folium.map.Marker(
        location=sigungu_dict[sigungu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 8pt">{sigungu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">경기도 시군구별 공원면적</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

In [18]:
map = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df['인당 공원면적'],      # 단계구분도로 보여줄 데이터
    columns=[df.index, df['인당 공원면적']],    # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',        # Colormap
    key_on='properties.name'    # 지도에서 조인할 항목
).add_to(map)
for sigungu_name in df.index:
    folium.map.Marker(
        location=sigungu_dict[sigungu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 8pt">{sigungu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">경기도 시군구별 인당 공원면적</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map